<a href="https://www.kaggle.com/code/matinmahmoudi/from-scratch-modified-perceptron-using-oop?scriptVersionId=201715568" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

class Perceptron:
    """
    Perceptron implemented from scratch using hinge loss and gradient descent.
    Loss and gradients are computed only for misclassified examples (negative margin).
    """
    
    def __init__(self, in_features, epochs=1000, learning_rate=0.01, tolerance=1e-6, verbose=0, random_state=42):
        np.random.seed(random_state)
        self.w = np.random.randn(in_features + 1, 1)  # Merged bias into weights by adding 1 to features
        self.epochs = epochs  # Number of iterations
        self.learning_rate = learning_rate  # Learning rate for gradient descent
        self.tolerance = tolerance  # Tolerance for early stopping
        self.verbose = verbose  # Verbosity level
        self.loss_history_ = []  # History of loss values
        self.accuracy_history_ = []  # History of accuracy values

    def _add_bias(self, X):
        """
        Add a bias term to the input data by appending a column of ones.
        X_bias = [1, x1, x2, ... xn]
        """
        return np.hstack([np.ones((X.shape[0], 1)), X])

    def fit(self, X, y):
        """
        Fit the perceptron model using gradient descent.
        Updates weights only based on examples where y * y_hat < 0 (misclassified).
        """
        X_bias = self._add_bias(X)  # Adding bias term to input features
        
        for epoch in range(self.epochs):
            y_hat = self.predict(X_bias)  # Raw predictions
            loss = self._loss(y, y_hat)  # Compute loss based only on misclassified points
            grad_w = self._grad(X_bias, y, y_hat)  # Compute gradients based on misclassified points
            
            self.w -= self.learning_rate * grad_w  # Gradient descent update
            self.loss_history_.append(loss)  # Track loss
            
            if self.verbose and epoch % 100 == 0:
                accuracy = self.score(X, y)
                print(f"Epoch {epoch}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")
            
            # Early stopping based on tolerance
            if epoch > 0 and abs(self.loss_history_[-2] - self.loss_history_[-1]) < self.tolerance:
                if self.verbose:
                    print(f"Early stopping at epoch {epoch}")
                break
        
        self.plot_learning_curve()

    def predict(self, X):
        """
        Predict raw scores (without thresholding).
        Prediction: y_hat = Xw
        """
        return X @ self.w
    
    def predict_class(self, X):
        """
        Predict class labels by applying a threshold of 0 to the output.
        Class prediction: y_pred = 1 if Xw >= 0 else -1
        """
        X_bias = self._add_bias(X)
        return np.where(self.predict(X_bias) >= 0, 1, -1)
    
    def score(self, X, y):
        """
        Calculate the accuracy of the model.
        Accuracy: (Number of correct predictions) / (Total predictions)
        """
        y_hat = self.predict(X)
        return self._accuracy(y, y_hat, t=0)
    
    def _accuracy(self, y, y_hat, t=0):
        """
        Accuracy calculation with thresholding at t=0.
        y_pred = 1 if y_hat >= t else -1
        """
        y_hat = np.where(y_hat < t, -1, 1)
        acc = np.sum(y == y_hat) / len(y)
        return acc
    
    def _loss(self, y, y_hat):
        """
        Hinge loss:
        L = mean(max(0, -y * y_hat))  # Only misclassified points (where y * y_hat < 0) contribute to the loss.
        """
        # Calculate loss only for misclassified examples
        return np.maximum(0, -y * y_hat).mean()
    
    def _grad(self, X, y, y_hat):
        """
        Gradient of hinge loss:
        ∇L = -mean(y * X) for misclassified points (where y * y_hat < 0)
        """
        # Calculate gradients only for misclassified examples (y * y_hat < 0)
        mask = y * y_hat < 0  # Only misclassified points contribute to gradient
        grad_w = (-y[mask, np.newaxis] * X[mask]).mean(axis=0).reshape(self.w.shape)
        return grad_w

    def plot_learning_curve(self):
        """
        Plot the learning curve for the model's loss over epochs.
        """
        if self.loss_history_:
            plt.plot(self.loss_history_)
            plt.title("Learning Curve")
            plt.xlabel("Epochs")
            plt.ylabel("Loss")
            plt.grid(True)
            plt.show()
    
    def __repr__(self):
        return f'Perceptron(epochs={self.epochs}, learning_rate={self.learning_rate}, tolerance={self.tolerance})'
